In [2]:
import numpy as np
from itertools import combinations

from functions import *

In [ ]:
def basis_set_generator(tot_sites, N_e_up, N_e_down): # Checked OK
    spin_up_basis = []
    spin_down_basis = []

    for comb_up in combinations(range(tot_sites), N_e_up):
        state = [0] * tot_sites
        for idx in comb_up:
            state[idx] = 1
        spin_up_basis.append(state)

    for comb_down in combinations(range(tot_sites), N_e_down):
        state = [0] * tot_sites
        for idx in comb_down:
            state[idx] = 1
        spin_down_basis.append(state)

    basis = []

    for up_state in spin_up_basis:
        for down_state in spin_down_basis:
            basis.append(up_state + down_state)
    
    return np.array(basis)


In [ ]:
tot_sites = 4
N_e_up = 2
N_e_down = 2

basis_set = basis_set_generator(tot_sites, N_e_up, N_e_down)

print(len(basis_set))
print(basis_set)